# APSC-5984 Lab 7: API and Database

## 0. Overview

API stands for Application Programming Interface. It is a set of tools meant to interact with other software or database. In this lab, we will use `requests` to interact with two USDA APIs. Later this week, we will also use `sqlite3` to practice building a database on our own.


In [ ]:
import pandas as pd
import numpy as np
import requests

## 1. USDA local food portal API

### Configuration

Before you can use the API, you need to get an API key and essentail parameters, such as the database you want to interact with and the query you want to run. Please treat the API key as a password and do not share it with others. Go to the [USDA local food portal](https://www.usdalocalfoodportal.com/fe/fregisterpublicapi) to register for an API key.

In [ ]:
API = "xxxxxxx" # fill in your API key
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "Referer": "https://www.google.com/",
}
params = {"apikey": API, "y": 37.221508, "x": -80.423857, "radius": 100}
base_url = "https://www.usdalocalfoodportal.com/api/farmersmarket/"

### Request responses

The responses are usually coded as 3-digit numbers. The most common ones are:

* 1xx - Informational response - the request was received, continuing process
* 2xx - Success - the request was successfully received, understood, and accepted
* 3xx - Redirection - further action needs to be taken in order to complete the request
* 4xx - Client error - the request contains bad syntax or cannot be fulfilled
* 5xx - Server error - the server failed to fulfill an apparently valid request

In most cases, you want to get a 2xx response. If you get a 4xx or 5xx response, you need to check your code and make sure you are using the API correctly.

In [ ]:
response = requests.get(base_url, headers=headers, params=params)
response

### Parsing JSON

The response from the API is usually in JSON format. JSON stands for JavaScript Object Notation. It is a lightweight data-interchange format. It is not easy for humans to read and write, hense we can call `.json()` and `pd.DataFrame()` to parse the response into a tabular format.

In [ ]:
json = response.json()
df = pd.DataFrame(json["data"])
df.head()

### Rearranging the data

The data might contain more information than you need. We can use Pandas techniques we learned before to rearrange the data.

In [ ]:
cols = [
    "listing_name",
    "brief_desc",
    "contact_name",
    "contact_phone",
    "media_website",
    "media_facebook",
    "media_twitter",
    "location_city",
    "location_state",
    "location_y",
    "location_x",
]

df.loc[:, cols]

## 2. Agricultural Resource Management Survey (ARMS) API

### API key

Go to [ARMS API](https://www.ers.usda.gov/developer/data-apis/arms-data-api/#apiForm) to obtain an API key for the USDA ARMS API.

In [ ]:
API = "xxxxxxx" # fill in your API key
params = {"api_key": API}

### Use `arms/year` to get the list of available years

This API has a lot of data. Based on the documentation https://www.ers.usda.gov/developer/data-apis/arms-data-api/#apiForm, we can use `arms/year` to get the list of available years.

In [ ]:
# arms/year
url = "https://api.ers.usda.gov/data/arms/year"
response = requests.get(url, headers=headers, params=params)
json_data = response.json()
json_data

### Use `arms/report` to get the list of available reports

To make sure we are using the API correctly, we can use `arms/report` to get the list of available reports. We will later need to use the correct report name to get the data.

In [ ]:
# get all report
url = "https://api.ers.usda.gov/data/arms/report"
response = requests.get(url, headers=headers, params=params)
js = response.json()
df = pd.DataFrame(response.json()["data"])
df

### Farm Business Income Statement

Let's say we want to get the farm business income statement for all farms in the US for the years 2008 to 2022. We can start with a single year, say 2018.

In [ ]:
params = {
    "api_key": API,
    "Year": 2018,
    "report": "Farm Business Income Statement",
}
url = "https://api.ers.usda.gov/data/arms/surveydata"
response = requests.get(url, headers=headers, params=params)
json_data = response.json()
data = pd.DataFrame(json_data["data"])
data

### Check the data

You can check the data by inspecting the unique values of each column of interest. For example, if you want to know any category other than "Operator Age", you can use the following code:

In [ ]:
data["category"].unique()

You may also want to know what `Production Specialty` was recorded as. You can combine `df.query()` and `df.unique()`.

In [ ]:
data.query("category == 'Production Specialty'").loc[:, "category_value"].unique()

We can check variables to know what attributes are available for that category.

In [ ]:
data.query("category == 'Production Specialty'").loc[:, "variable_name"].unique()

### Functions

It is a good idea to wrap up the code into functions so that you can reuse them later. Especially when we deal with the queries with repeated patterns, we can use functions to make the code more concise.

To design a function, first thing we can to define is the input and output. In our case, we provide API key and parameters (year, report name, etc.) as input and get a parsed dataframe as output.

In [ ]:
def query_data(year, API, report="Farm Business Income Statement"):
    # inputs
    params = {
        "api_key": API,
        "Year": year,
        "report": report,
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Referer": "https://www.google.com/",
    }
    url = "https://api.ers.usda.gov/data/arms/surveydata"
    # query
    response = requests.get(url, headers=headers, params=params)
    print("status code:", response.status_code)
    json_data = response.json()
    data = pd.DataFrame(json_data["data"])
    # parse output
    data = data.query("variable_name == 'Gross cash farm income'")
    return data

Validate the function

In [ ]:
data = query_data(2018, API)
data.head()

We can define another function to process the data we get from the API. In this example, we will do the following processing:

- Select the columns of interest
- calculate standard deviation of the income statement
- calculate 95% confidence interval of the standard deviation

In [ ]:
def post_process(data):
    cols = ["year", "category", "category_value", "estimate", "median", "rse"]
    datasub = data.loc[:, cols]
    datasub["se"] = datasub["estimate"] * (datasub["rse"] / 100)
    datasub["upper"] = datasub["estimate"] + datasub["se"] * 1.96  # 95% CI
    datasub["lower"] = datasub["estimate"] - datasub["se"] * 1.96  # 95% CI
    return datasub

Always check the results to make sure the function works as expected.

In [ ]:
data18 = query_data(2018, API)
data18 = post_process(data18)
data18.head()

### Query multiple years

With the functions we defined, it is now easier to query multiple years. We can use a for loop to iterate through the years and use the functions we defined to get the data. To avoid overloading the server, we can use `time.sleep()` to pause the code for one second between each query.

In [ ]:
import time

data = pd.DataFrame()
for year in range(2008, 2022):  # only 2008-2021
    print("Loading data for year", year)
    data_query = query_data(year, API)
    data_query = post_process(data_query)
    data = pd.concat([data, data_query], axis=0)
    time.sleep(1)

Check the result

In [ ]:
data

### Visualize the data

Now, let's focus on the two categories: `NASS Region` and `Production Specialty`. We will use ggplot2 (`plotnine` library) to visualize the income statement for each category across years.

In [ ]:
from plotnine import *
dataplot = data.query("category in ['NASS Region', 'Production Specialty']")
dataplot["group"] = dataplot["category"] + " - " + dataplot["category_value"]
dataplot

By region

In [ ]:
(
    ggplot(
        dataplot.query("category == 'NASS Region'"),
        aes(x="year", color="category_value", group="group"),
    )
    + geom_line(aes(y="estimate", color="category_value"))
    + geom_line(aes(y="median", color="category_value"))
    + geom_ribbon(aes(ymin="lower", ymax="upper", fill="category_value"), alpha=0.2)
    + theme(figure_size=(10, 10))
)

By production specialty. We only show the livestock categories.

In [ ]:
(
    ggplot(
        dataplot.query(
            "category == 'Production Specialty' and category_value in ['Dairy', 'Cattle', 'Hogs', 'Poultry']"
        ),
        aes(
            x="year",
            color="category_value",
            fill="category_value",
            group="category_value",
        ),
    )
    + geom_line(aes(y="estimate"))
    + geom_line(aes(y="median"))
    + geom_ribbon(aes(ymin="lower", ymax="upper"), alpha=0.2)
    # color theme
    + scale_color_brewer(type="qual", palette="Set2")
    + scale_fill_brewer(type="qual", palette="Set2")
    + theme(figure_size=(10, 10))
)

We can apply a log transformation to the data to make the plot more readable.

In [ ]:
import numpy as np

dataplot["log_estimate"] = np.log(dataplot["estimate"])
dataplot["log_upper"] = np.log(dataplot["upper"])
dataplot["log_lower"] = np.log(dataplot["lower"])
dataplot["log_median"] = np.log(dataplot["median"])
(
    ggplot(
        dataplot.query(
            "category == 'Production Specialty' and category_value in ['Dairy', 'Cattle', 'Hogs', 'Poultry']"
        ),
        aes(
            x="year",
            color="category_value",
            fill="category_value",
            group="category_value",
        ),
    )
    + geom_line(aes(y="log_estimate"), size=1)
    + geom_line(aes(y="log_median"))
    + geom_ribbon(aes(ymin="log_lower", ymax="log_upper"), alpha=0.2)
    + scale_color_brewer(type="qual", palette="Set2")
    + scale_fill_brewer(type="qual", palette="Set2")
    + theme(figure_size=(10, 10))
)


## 3. SQLite3

After we learn how to interact with an existing database through API, we can also build our own database using our own data. In this section, we will use `sqlite3` to build a database and query the data.


In [ ]:
import sqlite3

### Create a database

Creating a database is as simple as creating a file. We can use `sqlite3.connect()` to create a database file.

In [ ]:
conn = sqlite3.connect("demo.db") # conn stands for connection

Like we learned in the previous section, a database can contain multiple tables (or surveys, reports). To craete a table, we need to specify the name of the columns and the data type. We also need to specify the primary key, which is a unique identifier for each row. We can create a table named `users` with the following columns:

- `id` - INTEGER (PRIMARY KEY)
- `name` - TEXT
- `gender` - TEXT
- `age` - INTEGER

In [ ]:
cur = conn.cursor() # cur stands for cursor
cur.execute(
    """
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        gender TEXT,
        age INTEGER
    )
    """
)

Check the table columns

In [ ]:
cur.execute("PRAGMA table_info(users)")
cur.fetchall()

If you think the command is too complicated, we can use a function to simplify the process.

In [ ]:
def list_cols(cur, table):
    cur.execute("PRAGMA table_info(%s)" % table)
    return display([x[1:3] for x in cur.fetchall()])

list_cols(cur, "users")

### Save the database

We can use `conn.commit()` to save the changes to the database. Like how we deal with a file, we need to close `conn.close()` the database after we are done with it.

In [ ]:
conn.commit()
conn.close()

Or we can use a `with` statement to automatically close the database after we are done with it.

In [ ]:
with sqlite3.connect("demo.db") as conn:
    cur = conn.cursor()
    list_cols(cur, "users")

### Insert data

There are two ways to insert data into a table:

Provide information for all columns
- `INSERT INTO users VALUES (1, 'John', 'M', 20)`

Provide information for some columns
- `INSERT INTO users (name, gender) VALUES ('John', 'M')`

In [ ]:
cur.execute("INSERT INTO users VALUES (1, 'Mary', 'F', 25)")
cur.execute("INSERT INTO users (name) VALUES ('John')")

Check the data

In [ ]:
cur.execute("SELECT * FROM users")
cur.fetchall()

Or wrap it up in a function

In [ ]:
def print_table(cur, table):
    cur.execute("SELECT * FROM %s" % table)
    display(cur.fetchall())

print_table(cur, "users")

You can actually use `df.to_sql()` to insert data into a table. Parameters we need to consider:

- `if_exists`: If the table already exists, we can choose to `replace` the table, or `append` the data to the existing table.
- `index`: whether to include the index of the dataframe as a column in the table.

In [ ]:
df = pd.DataFrame(
    {
        "name": ["Camille", "Mike", "Jason", "Maria"],
        "gender": ["female", "male", "male", "female"],
        "age": [40, 25, 35, 20],
    }
)
df.to_sql("users", conn, if_exists="append", index=False)  # if_exists="replace"

In [ ]:
print_table(cur, "users")

### Add constraints to columns

You might notice that the gender values were not in a consistent format, which should either be [`M`, `F`] or [`Male`, `Female`]. We can use `CHECK` to add constraints to the columns.

Before re-creating the table, we need to drop the table first.

In [ ]:
cur.execute("DROP TABLE users")

Then create a new table with the constraints:

* id - INTEGER (PRIMARY KEY)
* name - TEXT - NOT NULL
* gender - TEXT - can only be either 'male' or 'female'
* age - INTEGER - must be in the range of 0 to 150
* weight - REAL - must be in the range of 0 to 300

In [ ]:
cur.execute(
    """
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        gender TEXT CHECK(gender IN ("male", "female")),
        age INTEGER CHECK(age >= 0 AND age <= 150),
        weight REAL CHECK(weight >= 0 AND weight <= 300)
    )
    """
)

Now, let's try to insert data again. We can start with expected values:

In [ ]:
cur.execute("INSERT INTO users VALUES (1, 'Mary', 'female', 25, 150)")

In [ ]:
print_table(cur, "users")

Let's try different exceptions:

In [ ]:
cur.execute("INSERT INTO users VALUES (2, 'Mary', 'f', 25, 150)")

In [ ]:
cur.execute("INSERT INTO users VALUES (2, 'Mary', 'female', -3, 200)")

By setting constraints to the columns, it is easier to ensure the data quality when the database is growing. Here is a complete code for creating a `user` table with data inserted.

In [ ]:
with sqlite3.connect("demo.db") as conn:
    cur = conn.cursor()
    cur.execute(
    """
    CREATE TABLE users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        gender TEXT CHECK(gender IN ("male", "female")),
        age INTEGER CHECK(age >= 0 AND age <= 150),
        weight REAL CHECK(weight >= 0 AND weight <= 300)
    )
    """)

    df = pd.DataFrame(
        {
            "name": ["Camille", "Mike", "Jason", "Maria"],
            "gender": ["female", "male", "male", "female"],
            "age": [40, 25, 35, 20],
            "weight": [150, 200, 180, 120],
        }
    )
    df.to_sql("users", conn, if_exists="append", index=False)
    print_table(cur, "users")

### Reference integrity

We already have a table `users` to define users' information. Now, let's create another table `walks` to record the walking activity.

In [ ]:
conn = sqlite3.connect("demo.db")
cur = conn.cursor()
cur.execute(
    """
    CREATE TABLE walks (
        id INTEGER PRIMARY KEY,
        user_id INTEGER NOT NULL,
        date TEXT NOT NULL,
        distance FLOAT NOT NULL,
        duration INTEGER NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users (id))
    """
)

You may notice that we set `user_id` as the foreign key. This means that the value of `user_id` must correspond to the value of `id` in the `users` table when we need to consider the relationship between the two tables. This is called a `reference integrity`. We can use `REFERENCES` to set the reference integrity.

The `walks` table include the information of the walking date, distance, and duration.Let's insert data into this `walks` table.

In [ ]:
data = pd.DataFrame(
    data={
        "user_id": [1, 1, 1, 2, 2, 3, 3, 4, 4, 4],
        "date": [
            "02-26-2023",
            "02-27-2023",
            "02-28-2023",
            "02-26-2023",
            "02-27-2023",
            "02-26-2023",
            "02-27-2023",
            "02-26-2023",
            "02-27-2023",
            "02-28-2023",
        ],
        "distance": [1.2, 1.5, 1.7, 2.2, 2.5, 3.2, 3.5, 4.2, 4.5, 4.7],
        "duration": [30, 40, 50, 60, 70, 80, 90, 100, 110, 120],
    }
)
data.to_sql("walks", conn, if_exists="append", index=False)

In [ ]:
print_table(cur, "walks")

Ok, now we have two tables with data inserted. We can use `JOIN` to put the data from two tables together.

In [ ]:
cur.execute(
    """
    SELECT * FROM walks JOIN users
    ON walks.user_id = users.id
    """
)
cur.fetchall()

You see that the user information was added to the `walks` table. This is similar to how we use `df.merge()` to combine two dataframes in Pandas.

In [ ]:
df_users = pd.read_sql("SELECT * FROM users", conn)
df_walks = pd.read_sql("SELECT * FROM walks", conn)
pd.merge(df_walks, df_users, left_on="user_id", right_on="id")

### SQlite3 VS. Pandas

Here we will put the major functionalities of SQLite3 and Pandas side by side to see how they compare.

#### Sorting

SQLite3

In [ ]:
cur.execute("SELECT * FROM users ORDER BY age DESC")  # or ASC
cur.fetchall()

Pandas

In [ ]:
df_users.sort_values(by="age", ascending=False)

#### Filtering

SQLite3

In [ ]:
cur.execute("SELECT age, gender FROM users WHERE age > 30")
cur.fetchall()

Pandas

In [ ]:
df_users.loc[:, ["age", "gender"]].query("age > 30")

SQLite3

In [ ]:
cur.execute(
    """
    SELECT name, weight FROM users
    WHERE name LIKE '%m%'
    OR name LIKE '%n%'
    """
)
cur.fetchall()

Pandas

In [ ]:
df_users.loc[:, ["name", "weight"]].query(
    """
    name.str.upper().str.contains('M') |\
    name.str.upper().str.contains('N')
    """
)

#### Grouping

SQLite3

In [ ]:
cur.execute(
    """
    SELECT user_id, avg(distance), sum(duration), count(distance)
    FROM walks GROUP BY user_id
    """
)
cur.fetchall()

Pandas

In [ ]:
df_walks.groupby("user_id").aggregate(
    distance_mean=("distance", "mean"),
    duration_sum=("duration", "sum"),
    distance_count=("distance", "count"),
)

### sqlite_master

`sqliet_master` is a system table that contains the information of all tables in the database. We can use `SELECT * FROM sqlite_master` to get the information of all tables. The output will look like this:

* `type`: the type of the object. In this case, it is `table`.
* `name`: the name of the table.
* `tbl_name`: the name of the table.
* `rootpage`: the page number of the root b-tree page for the table.
* `sql`: the SQL statement used to create the table.

In [ ]:
cur.execute("SELECT * FROM sqlite_master")
cur.fetchall()

Or simply list all the tables in the database.

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
cur.fetchall()

### Collection of functions

In [ ]:
def list_cols(cur, table):
    cur.execute("PRAGMA table_info(%s)" % table)
    cols = [x[1:3] for x in cur.fetchall()]
    return display(cols)

def print_table(cur, table):
    cur.execute("SELECT * FROM %s" % table)
    output = cur.fetchall()
    display(output)

def clean_table(cur, table):
    cur.execute("DELETE FROM %s" % table)

def drop_table(cur, table):
    cur.execute("DROP TABLE %s" % table)